<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modules

In [1]:
import os
import sys
from google.colab import drive

ModuleNotFoundError: No module named 'google'

In [ ]:


# Mount Google Drive to the root location with force_remount
drive.mount('/content/drive', force_remount=True)

# Assuming 'BOE' folder is in 'MyDrive' and already shared
BOE_path = '/content/drive/MyDrive/BOE/bank_of_england/data'

# Now you (and others with access) can work with files in this directory
# For example, you can list the contents:
print(os.listdir(BOE_path))

In [ ]:

sys.path.append('/content/drive/My Drive/bank_of_england/src')

from preprocessing import preprocess_pipeline

# Example usage
sample_text = "JP Morgan's earnings increased by 20% in Q4 2024."
processed_text = preprocess_pipeline(sample_text)
print(processed_text)  # Output: "jpmorgan earning increase q4"


In [ ]:
import pdfplumber
import pandas as pd
import re

# Path to the JPMorgan transcript PDF (replace with actual path)
pdf_path = "path_to_your_JPMorgan_PDF.pdf"

# Function to extract and clean text from PDF
def extract_clean_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text_data = []
        
        # Skip first page and process remaining pages
        for i, page in enumerate(pdf.pages[1:]):  # Skipping first page
            text = page.extract_text()
            if text:
                # Remove headers and footers
                text = re.sub(r"JPMorgan Chase & Co\..*", "", text)  # Remove headers
                text = re.sub(r"Page \d+ of \d+", "", text)  # Remove footers
                text = re.sub(r"Copyright © \d{4} JPMorgan Chase & Co\.", "", text)  # Remove copyright notices
                
                # Remove excessive spaces and blank lines
                text = re.sub(r"\n\s*\n", "\n", text).strip()
                
                text_data.append(text)

        # Join extracted text
        full_text = "\n".join(text_data)

        # Remove disclaimers based on actual text patterns
        disclaimer_match = re.search(
            r"(Disclaimer\s+This document contains forward-looking statements.*?JPMorgan Chase & Co\. does not undertake to update any forward-looking statements)",
            full_text,
            re.IGNORECASE | re.DOTALL
        )
        if disclaimer_match:
            full_text = full_text[:disclaimer_match.start()].strip()  # Remove disclaimer section

        return full_text

# Extract and clean the text
cleaned_text = extract_clean_text(pdf_path)

# Save cleaned text to CSV
csv_path = "cleaned_jpmorgan_transcript.csv"
df = pd.DataFrame({"Text": cleaned_text.split("\n")})  # Convert to DataFrame
df.to_csv(csv_path, index=False)

# Print confirmation
print(f"Cleaned transcript saved to {csv_path}")
